# Self-Driving Car Engineer Nanodegree


## Project: **Vehicle Detection and Tracking** 
***

**_Project Objective:_**

Take video captures from a car-mounted camera and output video with bounding boxes drawn around surrounding vehicles. This module will be added to the `lane_finder` module, ultimately to be used to better train an autonomous driving model.

**_Implementation Pipeline:_**

1. Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier.
2. Optionally, apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.
    1. Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
3. Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
4. Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
5. Estimate a bounding box for vehicles detected.
***

## Feature Extraction

## Load Data

## Build Classifier

## Sliding Window

## Test Classifier

## Sliding Window - Revisited

## Process Videos